In [30]:
import yaml
import numpy as np
import pandas as pd
import math

In [13]:
filename = 'testrig_collected_data_202204271357.yaml'

In [49]:
data_keys = ['base_left_marker_pose',
             'base_right_marker_pose',
             'wrist_inside_marker_pose',
             'wrist_top_marker_pose',
             'shoulder_marker_pose']

def get_dict(filename):
    with open(filename,'r') as file:
        data = yaml.safe_load(file)
    return data

def get_avg_pos_vals(data):
    avg_pose_sums = {}
    null_cnt = {}
    for key in data_keys:
        avg_pose_sums[key] = np.zeros(3)
        null_cnt[key] = 0
    for d in data:
        for key in data_keys:
            m = np.array(d['camera_measurements'][key])
            try:
                pose = m[:3,3]
                avg_pose_sums[key] = avg_pose_sums[key] + pose
            except:
                null_cnt[key] = null_cnt[key] + 1
    for key in data_keys:
        avg_pose_sums[key] = avg_pose_sums[key]/(len(data)-null_cnt[key])
    return avg_pose_sums

def euclidean_error(r1, r2):
    p1 = r1[:3, 3]
    p2 = r2[:3, 3]
    dist1 = math.sqrt(math.pow(p1[0],2) + math.pow(p1[1],2) + math.pow(p1[2],2))
    dist2 = math.sqrt(math.pow(p2[0],2) + math.pow(p2[1],2) + math.pow(p2[2],2))
#     dist = np.linalg.norm(r1[:3, 3] - r2[:3, 3])
    return float(abs(dist1-dist2))

In [50]:
data = get_dict(filename)

In [51]:
get_avg_pos_vals(data)

{'base_left_marker_pose': array([ 0.01560918, -0.03676896,  0.97180282]),
 'base_right_marker_pose': array([ 0.08429622, -0.03563762,  0.9740998 ]),
 'shoulder_marker_pose': array([ 0.09266814, -0.09151441,  0.97548074]),
 'wrist_inside_marker_pose': array([ 0.00508762, -0.09562531,  0.97365567]),
 'wrist_top_marker_pose': array([ 0.0466689 , -0.09545316,  0.97414828])}

In [52]:
data[0]['camera_measurements'].keys()

['shoulder_marker_pose',
 'd435i_acceleration',
 'wrist_inside_marker_pose',
 'wrist_top_marker_pose',
 'base_right_marker_pose',
 'base_left_marker_pose']

In [53]:
m = np.array(data[0]['camera_measurements'][data_keys[0]])
m

array([[ 0.99847575, -0.00292909, -0.05511444,  0.01565563],
       [-0.00900108, -0.99386336, -0.11024792, -0.03676937],
       [-0.05445329,  0.11057596, -0.99237483,  0.97191084],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [54]:
nominal_poses = get_dict('testrig_marker_info.yaml')

In [55]:
mn = nominal_poses['testrig_aruco_marker_info'][data_keys[0]]
mn = np.array(mn)
mn

array([[ 1.     ,  0.     ,  0.     ,  0.009  ],
       [ 0.     , -1.     ,  0.     , -0.06418],
       [ 0.     ,  0.     , -1.     ,  0.97551],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [56]:
euclidean_error(m,mn)

0.004928270424978343